In [55]:
from model_handler import OPENAI_API_KEY

import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
import nltk

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [70]:
loader = DirectoryLoader('./texts/', glob='**/*.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
docsearch = FAISS.from_documents(texts, embeddings)

In [71]:
from uuid import uuid4
import pinecone

pinecone.

index = pinecone.GRPCIndex("openai-chatpt")

print(type(texts))
batch_limit = 100

AttributeError: module 'pinecone' has no attribute 'GRPCIndex'

In [ ]:
if len(texts) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeds = embeddings.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []

In [72]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
docsearch = FAISS.from_documents(texts, embeddings)
# llm = OpenAI(openai_api_key=OPENAI_API_KEY)
# qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())
# query = "Tell me about yourself?"

# qa.run(query)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).


<class 'langchain.vectorstores.faiss.FAISS'>


In [63]:
# import pinecone
# import numpy as np

# std_key = "681591b9-096b-4da3-8df2-9de5f89fba34"
# pinecone.init(api_key=std_key)

# all_embeddings = np.array(embeddings.get_embeddings())

# normalized_vectors = all_embeddings / np.linalg.norm(all_embeddings, axis=1, keepdims=True)

# pinecone_index = pinecone.Index(index_name="openai-chatpt")
# pinecone_index.upsert(ids=["159632"], vectors=normalized_vectors)



AttributeError: 'RetrievalQA' object has no attribute 'get_embeddings'

In [ ]:
import pinecone
std_key = "681591b9-096b-4da3-8df2-9de5f89fba34"

index_name = 'langchain-retrieval-augmentation'
pinecone.init(api_key=std_key)

if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, metric='cosine')  


In [ ]:
from tqdm.auto import tqdm
from uuid import uuid4

index = pinecone.GRPCIndex(index_name)
batch_limit = 100

texts_array = []
metadatas = []

for i, record in enumerate(tqdm(documents)):

    record_metadatas = [{"chunk": j, "text": text} for j, text in enumerate(text_splitter)]
   
    texts_array = [].extend(texts)
    metadatas.extend(record_metadatas)
    
    texts.extend(text_splitter)
    metadatas.extend(record_metadatas)

    if len(texts_array) >= batch_limit:
        ids = [str(uuid4()) for _ in range(len(texts_array))]
        embeds = embeddings.embed_documents(texts_array)
        index.upsert(vectors=zip(ids, embeds, metadatas))
        texts = []
        metadatas = []

if len(texts) > 0:
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeds = embeddings.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeds, metadatas))
